In [2]:
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

Build 3D dataset

In [3]:
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

# manually Projection PCA

    find d priciple vectors with most variance (if map data on it) and then transform data on new space
    use svd to find vectors with mathematic

In [4]:
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]

In [5]:
m, n = X.shape

S = np.zeros(X_centered.shape)
S[:n, :n] = np.diag(s) # qotri

In [6]:
S


array([[6.77645005, 0.        , 0.        ],
       [0.        , 2.82403671, 0.        ],
       [0.        , 0.        , 0.78116597],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.

In [7]:
np.allclose(X_centered, U.dot(S).dot(Vt))
# Returns True if two arrays are element-wise equal within a tolerance

True

In [8]:
W2 = Vt.T[:, :2]
X2D = X_centered.dot(W2)
X2D_using_svd = X2D

# PCA  using scikit learn
takes care of mean centering

In [11]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 2)

# find PCs
# transford them
X2D = pca.fit_transform(X)

In [12]:
X2D[:5]

array([[ 1.26203346,  0.42067648],
       [-0.08001485, -0.35272239],
       [ 1.17545763,  0.36085729],
       [ 0.89305601, -0.30862856],
       [ 0.73016287, -0.25404049]])

In [13]:
X2D_using_svd[:5]

array([[-1.26203346, -0.42067648],
       [ 0.08001485,  0.35272239],
       [-1.17545763, -0.36085729],
       [-0.89305601,  0.30862856],
       [-0.73016287,  0.25404049]])

if train severall time it may have different Principle Components. in general difference is that they just flipped(negative of each other)

In [14]:
np.allclose(X2D, X2D_using_svd), np.allclose(X2D, -X2D_using_svd)

(False, True)

In [17]:
# Recover the 3D points projected on the plane (PCA 2D subspace)

X3D_inv = pca.inverse_transform(X2D)

there was some loss of information during the projection step
so the recovered 3D points are not exactly equal to the original 3D points

In [19]:
np.allclose(X, X3D_inv)

False

In [24]:
# reconstruction error

np.mean(np.sum(np.square(X3D_inv - X) , axis = 1))

0.010170337792848549

In [26]:
X3D_inv_using_svd = X2D_using_svd.dot(Vt[:2, :])

In [27]:
# sklearn tack care of mean and reversing it

np.allclose(X3D_inv_using_svd, X3D_inv - pca.mean_)

True

In [28]:
pca.mean_

array([0.02406745, 0.20932515, 0.07155422])

In [29]:
#principle components

pca.components_

array([[-0.93636116, -0.29854881, -0.18465208],
       [ 0.34027485, -0.90119108, -0.2684542 ]])

In [30]:
# principle componets manually

Vt

array([[ 0.93636116,  0.29854881,  0.18465208],
       [-0.34027485,  0.90119108,  0.2684542 ],
       [-0.08626012, -0.31420255,  0.94542898]])

In [32]:
# explained variance: portion of dataset's variance for each PC

pca.explained_variance_ratio_
# only 1.1% remain for 3th PC so we lost little information

array([0.84248607, 0.14631839])

In [34]:
#  compute the explained variance ratio using the SVD approach (recall that s is the diagonal of the matrix S)

np.square(s)/np.square(s).sum()

array([0.84248607, 0.14631839, 0.01119554])

### chosing Number for dimention reduction
#### 2d or 3d for visualization, more than threshold for variance

In [36]:
# create dataset

from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.target = mnist.target.astype(np.uint8)

# split train and test

from sklearn.model_selection import train_test_split

X = mnist["data"]
y = mnist["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [38]:
pca = PCA()
pca.fit(X_train)

cumsum = np.cumsum(pca.explained_variance_ratio_) # accumulated sum of prev elements on each column

# 1. can set d 
d = np.argmax(cumsum >=0.95) +1



cumsum, np.argmax(cumsum >=0.95) , d # 784 ==> 154 !!!

(array([0.09719832, 0.16875148, 0.23046024, 0.28447767, 0.33353621,
        0.37656401, 0.40934646, 0.43819275, 0.46567853, 0.48924485,
        0.51032629, 0.5307285 , 0.54778858, 0.56465048, 0.58041792,
        0.59534958, 0.60862878, 0.62147783, 0.63334578, 0.64479193,
        0.65545804, 0.66555448, 0.67514241, 0.68416896, 0.69296211,
        0.70131513, 0.70939893, 0.71727437, 0.72468736, 0.73157212,
        0.73812949, 0.74459959, 0.75058197, 0.75643475, 0.76210811,
        0.7675608 , 0.77261474, 0.77750626, 0.78230885, 0.78696883,
        0.79152081, 0.79597374, 0.80014325, 0.80411726, 0.80795962,
        0.81171266, 0.81533146, 0.81882001, 0.8221978 , 0.82541301,
        0.82859763, 0.83168676, 0.83465474, 0.83752128, 0.84034773,
        0.84303834, 0.84571815, 0.84828932, 0.85083357, 0.85329543,
        0.85569464, 0.8580644 , 0.86035289, 0.86256245, 0.86468703,
        0.86673787, 0.86875875, 0.87071732, 0.87263867, 0.87451472,
        0.87637902, 0.87817437, 0.87993644, 0.88

In [39]:
# 2. can set ncomponents as a ration of varianace
pca = PCA(n_components = 0.95)
X_reduce = pca.fit_transform(X_train)

In [40]:
pca = PCA(n_components = 154)
X_reduced = pca.fit_transform(X_train)
X_recovered = pca.inverse_transform(X_reduced)

# Randomized PCA
#### y finds an approximation of the first d principal components
#### svd_solver to randomized

In [41]:
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced = rnd_pca.fit_transform(X_train)

# Incremental PCA
#### split TS into mini-batches --> good for PCA online

In [42]:
from sklearn.decomposition import IncrementalPCA

n_batches = 100

inc_pca = IncrementalPCA(n_components = 154)

for X_batch in np.array_split(X_train, n_batches): # array .split
   inc_pca.partial_fit(X_batch)  # partial_fit

X_reduce = inc_pca.transform(X_train)

### using memmap

#### stored in a binary file on disk as if it were entirely in memory
#### loads only the data it needs in memory

In [43]:
filename = "my_mnist.data"
m, n = X_train.shape

X_mm = np.memmap(filename, dtype='float32', mode='write', shape=(m, n))
X_mm[:] = X_train
 
#Now deleting the memmap() object will trigger its Python finalizer, which ensures that the data is saved to disk    
del X_mm

In [44]:
X_mm = np.memmap(filename, dtype="float32", mode="readonly", shape=(m, n))

batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)  # fit

IncrementalPCA(batch_size=525, n_components=154)

### time complexity

In [45]:
import time

for n_components in (2, 10, 154):
    print("n_components =", n_components)
    regular_pca = PCA(n_components=n_components)
    inc_pca = IncrementalPCA(n_components=n_components, batch_size=500)
    rnd_pca = PCA(n_components=n_components, random_state=42, svd_solver="randomized")

    for pca in (regular_pca, inc_pca, rnd_pca):
        t1 = time.time()
        pca.fit(X_train)
        t2 = time.time()
        print("    {}: {:.1f} seconds".format(pca.__class__.__name__, t2 - t1))

n_components = 2
    PCA: 2.5 seconds
    IncrementalPCA: 66.2 seconds
    PCA: 3.9 seconds
n_components = 10
    PCA: 6.6 seconds
    IncrementalPCA: 78.6 seconds
    PCA: 3.4 seconds
n_components = 154
    PCA: 25.2 seconds
    IncrementalPCA: 120.4 seconds
    PCA: 25.5 seconds


### different size of instances : pca vs random pca

In [ ]:
times_rpca = []
times_pca = []
sizes = [1000, 10000, 20000, 30000, 40000, 50000, 70000, 100000, 200000, 500000]
for n_samples in sizes:
    X = np.random.randn(n_samples, 5)
    pca = PCA(n_components = 2, svd_solver="randomized", random_state=42)
    t1 = time.time()
    pca.fit(X)
    t2 = time.time()
    times_rpca.append(t2 - t1)
    pca = PCA(n_components = 2)
    t1 = time.time()
    pca.fit(X)
    t2 = time.time()
    times_pca.append(t2 - t1)

plt.plot(sizes, times_rpca, "b-o", label="RPCA")
plt.plot(sizes, times_pca, "r-s", label="PCA")
plt.xlabel("n_samples")
plt.ylabel("Training time")
plt.legend(loc="upper left")
plt.title("PCA and Randomized PCA time complexity ")

### various feature performance: pca vs random pca

In [ ]:
times_rpca = []
times_pca = []
sizes = [1000, 2000, 3000, 4000, 5000, 6000]
for n_features in sizes:
    X = np.random.randn(2000, n_features)
    pca = PCA(n_components = 2, random_state=42, svd_solver="randomized")
    t1 = time.time()
    pca.fit(X)
    t2 = time.time()
    times_rpca.append(t2 - t1)
    pca = PCA(n_components = 2)
    t1 = time.time()
    pca.fit(X)
    t2 = time.time()
    times_pca.append(t2 - t1)

plt.plot(sizes, times_rpca, "b-o", label="RPCA")
plt.plot(sizes, times_pca, "r-s", label="PCA")
plt.xlabel("n_features")
plt.ylabel("Training time")
plt.legend(loc="upper left")
plt.title("PCA and Randomized PCA time complexity ")

# Kernel PCA

In [47]:
from sklearn.datasets import make_swiss_roll
X, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)


In [50]:
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

In [52]:
# which kernel?

# 1.grid search

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

y = t > 6.9

clf = Pipeline([
        ("kpca", KernelPCA(n_components=2)),
        ("log_reg", LogisticRegression(solver="lbfgs"))
    ])

param_grid = [{
        "kpca__gamma": np.linspace(0.03, 0.05, 10),
        "kpca__kernel": ["rbf", "sigmoid"]
    }]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kpca', KernelPCA(n_components=2)),
                                       ('log_reg', LogisticRegression())]),
             param_grid=[{'kpca__gamma': array([0.03      , 0.03222222, 0.03444444, 0.03666667, 0.03888889,
       0.04111111, 0.04333333, 0.04555556, 0.04777778, 0.05      ]),
                          'kpca__kernel': ['rbf', 'sigmoid']}])

In [53]:
print(grid_search.best_params_)

{'kpca__gamma': 0.043333333333333335, 'kpca__kernel': 'rbf'}


In [54]:
# 2. reconstruction error

## use pre-image error for kernels instead of square

rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.0433,
 fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)

In [55]:
from sklearn.metrics import mean_squared_error

mean_squared_error(X, X_preimage)

1.782416297453034e-26

# LLE Local linear embedding (manifold Learning)

In [56]:
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)
